In [1]:
import torch

In [2]:
torch.manual_seed(6)
# inputs = torch.rand(6, 3)
inputs = torch.tensor(
    [
        [0.43, 0.15, 0.89],
        [0.55, 0.78, 0.66],
        [0.57, 0.85, 0.64],
        [0.22, 0.58, 0.33],
        [0.77, 0.25, 0.10],
        [0.05, 0.80, 0.55]
    ]
)
inputs

tensor([[0.4300, 0.1500, 0.8900],
        [0.5500, 0.7800, 0.6600],
        [0.5700, 0.8500, 0.6400],
        [0.2200, 0.5800, 0.3300],
        [0.7700, 0.2500, 0.1000],
        [0.0500, 0.8000, 0.5500]])

In [3]:
query = inputs[1]
attn_scores_2 = torch.zeros(1, inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[0][i] = torch.dot(x_i, query)
print(attn_scores_2)

tensor([[0.9409, 1.3465, 1.3989, 0.7912, 0.6845, 1.0145]])


In [4]:
attn_scores_2.shape

torch.Size([1, 6])

In [5]:
# Normalize the attention scores
attn_scores_2 = torch.softmax(attn_scores_2, dim=-1) # Normalize along the last dimension, which is columns
# so that the values in the row sum to 1
print(attn_scores_2.sum())

tensor(1.)


In [6]:
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_scores_2[i] * x_i
print(context_vec_2)

RuntimeError: The size of tensor a (6) must match the size of tensor b (3) at non-singleton dimension 0

In [ ]:
# Compute the attention scores for all input vectors
attn_scores = torch.empty(inputs.shape[0], inputs.shape[0])
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)
print(attn_scores)

tensor([[0.9995, 0.9409, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9409, 1.3465, 1.3989, 0.7912, 0.6845, 1.0145],
        [0.9422, 1.3989, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.7912, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.6845, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0145, 1.0605, 0.6565, 0.2935, 0.9450]])


In [10]:
attn_scores = inputs @ inputs.T # @ is matrix multiplication
print(attn_scores)

tensor([[0.9995, 0.9409, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9409, 1.3465, 1.3989, 0.7912, 0.6845, 1.0145],
        [0.9422, 1.3989, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.7912, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.6845, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0145, 1.0605, 0.6565, 0.2935, 0.9450]])


In [11]:
attn_weights = torch.softmax(attn_scores, dim=-1)
attn_weights

tensor([[0.2104, 0.1984, 0.1987, 0.1246, 0.1224, 0.1455],
        [0.1472, 0.2209, 0.2328, 0.1268, 0.1139, 0.1585],
        [0.1415, 0.2234, 0.2367, 0.1264, 0.1128, 0.1592],
        [0.1451, 0.1989, 0.2067, 0.1478, 0.1276, 0.1739],
        [0.1533, 0.1923, 0.1984, 0.1373, 0.1887, 0.1301],
        [0.1406, 0.2063, 0.2160, 0.1442, 0.1003, 0.1925]])

In [12]:
attn_weights.shape

torch.Size([6, 6])

In [13]:
attn_weights[0].sum()

tensor(1.0000)

In [14]:
all_context_vecs = attn_weights @ inputs
all_context_vecs

tensor([[0.4418, 0.5745, 0.5788],
        [0.4410, 0.6210, 0.5661],
        [0.4412, 0.6256, 0.5654],
        [0.4291, 0.6094, 0.5499],
        [0.4667, 0.5725, 0.5260],
        [0.4157, 0.6284, 0.5631]])

In [15]:
torch.manual_seed(123)

d_in = inputs.shape[1]
d_out = 2
W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_key = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad = False)

In [17]:
W_query

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])

In [16]:
x_2 = inputs[1]
query_2 = x_2 @ W_query
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value
print(query_2)

tensor([0.4080, 1.3931])


In [18]:
keys = inputs @ W_key
values = inputs @ W_value
print(keys.shape)
print(values.shape)

torch.Size([6, 2])
torch.Size([6, 2])


In [19]:
# Compute the attention scores for input vector x_2
keys_2 = keys[1]
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.6737)


In [20]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2148, 1.6737, 1.7321, 1.0325, 0.5332, 1.4768])


In [21]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1533, 0.2120, 0.2209, 0.1347, 0.0946, 0.1844])


In [22]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.2976, 0.8081])


In [26]:
import torch.nn as nn
class SelfAttention_v1(nn.Module):
    def __init__(self, d_in, d_out):
        super().__init__() # Initialize the base class
        self.W_query = nn.Parameter(torch.rand(d_in, d_out)) # Random numbers from a uniform distribution on the interval [0, 1)
        self.W_key = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1) # Compute the attention weights
        context_vec = attn_weights @ values # Compute the context vectors
        return context_vec
    
class SelfAttention_v2(nn.Module):
    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias) # nn.Linear has an optimized weights initialization scheme TODO Understandy why?
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x) # nn.Linear abstracts the matrix multiplication and bias addition
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec


In [27]:
torch.manual_seed(789)

sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0719,  0.0726],
        [-0.0728,  0.0716],
        [-0.0728,  0.0716],
        [-0.0739,  0.0698],
        [-0.0742,  0.0692],
        [-0.0734,  0.0706]], grad_fn=<MmBackward0>)


In [30]:
sa_v2.W_query.weight

Parameter containing:
tensor([[ 0.3161,  0.4568,  0.5118],
        [-0.1683, -0.3379, -0.0918]], requires_grad=True)

In [34]:
sa_v1 = SelfAttention_v1(d_in, d_out)
sa_v1.W_query = torch.nn.Parameter(sa_v2.W_query.weight.T)
sa_v1.W_key = torch.nn.Parameter(sa_v2.W_key.weight.T)
sa_v1.W_value = torch.nn.Parameter(sa_v2.W_value.weight.T)
print(sa_v1(inputs))

tensor([[-0.0719,  0.0726],
        [-0.0728,  0.0716],
        [-0.0728,  0.0716],
        [-0.0739,  0.0698],
        [-0.0742,  0.0692],
        [-0.0734,  0.0706]], grad_fn=<MmBackward0>)


## Causal (masked) Attention: hide the future words

In [ ]:
context_length = attn_scores.shape[0]
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1).bool() # Upper triangular matrix
mask

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [55]:
masked = attn_scores.masked_fill(mask, -torch.inf)
masked

tensor([[0.9995,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.9409, 1.3465,   -inf,   -inf,   -inf,   -inf],
        [0.9422, 1.3989, 1.4570,   -inf,   -inf,   -inf],
        [0.4753, 0.7912, 0.8296, 0.4937,   -inf,   -inf],
        [0.4576, 0.6845, 0.7154, 0.3474, 0.6654,   -inf],
        [0.6310, 1.0145, 1.0605, 0.6565, 0.2935, 0.9450]])

In [40]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
attn_weights

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4288, 0.5712, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2618, 0.3615, 0.3767, 0.0000, 0.0000, 0.0000],
        [0.2199, 0.2749, 0.2825, 0.2228, 0.0000, 0.0000],
        [0.1832, 0.2151, 0.2199, 0.1695, 0.2122, 0.0000],
        [0.1488, 0.1951, 0.2016, 0.1515, 0.1172, 0.1858]])

In [51]:
# Masking additional attention weights with dropout
torch.manual_seed(123)
dropout = nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example)) 

tensor([[2., 2., 2., 2., 2., 2.],
        [0., 2., 0., 0., 0., 0.],
        [0., 0., 2., 0., 2., 0.],
        [2., 2., 0., 0., 0., 2.],
        [2., 0., 0., 0., 0., 2.],
        [0., 2., 0., 0., 0., 0.]])


In [52]:
dropout(example).sum() # Scale the output by 1/(1-p) to maintain the expected value of the input tensor

tensor(38.)

In [53]:
print(dropout(attn_weights))

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.1424, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.7231, 0.7534, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.5498, 0.5649, 0.0000, 0.0000, 0.0000],
        [0.3665, 0.4303, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3903, 0.4032, 0.0000, 0.2344, 0.3716]])


In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
batch.shapea

torch.Size([2, 6, 3])

In [56]:
class CasualAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, qkv_bias=False):
        super().__init__() # Initialize the base class
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1))

    def forward(self, x):
        bz, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        attn_scores = queries @ keys.transpose(1, 2) # keys shape before transpose is (bz, num_tokens, d_out)
        masked = attn_scores.masked_fill_(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf) # [:num_tokens, :num_tokens] is a slicing operation to extract a submatrix
        attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [58]:
torch.manual_seed(123)
context_length = batch.shape[1]
casual_attn = CasualAttention(d_in, d_out, context_length, 0.0)
context_vec = casual_attn(batch)
context_vec.shape

torch.Size([2, 6, 2])

## Multi-head causal attention mechanism

In [59]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList([CasualAttention(d_in, d_out, context_length, dropout, qkv_bias) for _ in range(num_heads)])
    
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [62]:
num_heads = 3
heads = nn.ModuleList([CasualAttention(d_in, d_out, context_length, 0.5) for _ in range(num_heads)])
heads

ModuleList(
  (0-2): 3 x CasualAttention(
    (W_query): Linear(in_features=3, out_features=2, bias=False)
    (W_key): Linear(in_features=3, out_features=2, bias=False)
    (W_value): Linear(in_features=3, out_features=2, bias=False)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [63]:
torch.manual_seed(123)
context_length = batch.shape[1] # Number of tokens in the input sequence
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.5, num_heads=2)
context_vec = mha(batch)
context_vec

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5704,  0.0276,  0.5740,  0.2991],
         [-0.6191, -0.0493,  0.6110,  0.3699],
         [-0.5593, -0.0741,  0.5410,  0.3475],
         [-0.5459, -0.0898,  0.5269,  0.3339],
         [-0.5243, -0.1012,  0.5030,  0.3416]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5704,  0.0276,  0.5740,  0.2991],
         [-0.6191, -0.0493,  0.6110,  0.3699],
         [-0.5593, -0.0741,  0.5410,  0.3475],
         [-0.5459, -0.0898,  0.5269,  0.3339],
         [-0.5243, -0.1012,  0.5030,  0.3416]]], grad_fn=<CatBackward0>)

In [64]:
context_vec.shape

torch.Size([2, 6, 4])

In [75]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads) == 0, "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer("mask", torch.triu(torch.ones(context_length, context_length), diagonal=1)) # diagonal=1 refers to the diagonal above the main diagonal

    def forward(self, x):
        bz, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)
        keys = keys.view(bz, num_tokens, self.num_heads, self.head_dim) # Split the matrix by adding a num_heads dimension
        queries = queries.view(bz, num_tokens, self.num_heads, self.head_dim)
        values = values.view(bz, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1,2) # Transpose from shape (bz, num_tokens, num_heads, head_dim) to (bz, num_heads, num_tokens, head_dim)
        queries = queries.transpose(1,2)
        values = values.transpose(1,2)
        print("keys shape:", keys.shape)
        print("queries shape:", queries.shape)
        print("keys transpose shape:", keys.transpose(2, 3).shape)
        attn_scores = queries @ keys.transpose(2, 3) # Perform the matrix multiplication bwteen the last two dimensions for each head
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        context_vec = (attn_weights@values).transpose(1,2) # Transpose back to (bz, num_tokens, num_heads, head_dim)
        context_vec = context_vec.contiguous().view(bz, num_tokens, self.d_out) # Combine the num_heads and head_dim dimensions
        # Contiguous() is used to ensure that the tensor is stored in a contiguous block of memory
        context_vec = self.out_proj(context_vec) # Apply the output projection to ensure that the output has the disired output dimensionality
        return context_vec

In [66]:
# Illustrate the batched matrix multiplication
a = torch.rand(1, 2, 3, 4) #bz=1, num_heads=2, num_tokens=3, head_dim=4
a

tensor([[[[0.0772, 0.3565, 0.1479, 0.5331],
          [0.4066, 0.2318, 0.4545, 0.9737],
          [0.4606, 0.5159, 0.4220, 0.5786]],

         [[0.9455, 0.8057, 0.6775, 0.6087],
          [0.6179, 0.6932, 0.4354, 0.0353],
          [0.1908, 0.9268, 0.5299, 0.0950]]]])

In [69]:
print(a @ a.transpose(2, 3)) # Matrix multiplication between the last two dimensions
print((a @ a.transpose(2, 3)).shape)

tensor([[[[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3738, 1.0621],
          [0.5903, 1.0621, 0.9911]],

         [[2.3727, 1.4592, 1.3439],
          [1.4592, 1.0531, 0.9944],
          [1.3439, 0.9944, 1.1851]]]])
torch.Size([1, 2, 3, 3])


In [71]:
first_head = a[0,0,:,:]
first_res = first_head @ first_head.T
print("First head result:\n", first_res)

print("\n")
second_head = a[0,1,:,:]
second_res = second_head @ second_head.T
print("Second head result:\n", second_res)

First head result:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3738, 1.0621],
        [0.5903, 1.0621, 0.9911]])


Second head result:
 tensor([[2.3727, 1.4592, 1.3439],
        [1.4592, 1.0531, 0.9944],
        [1.3439, 0.9944, 1.1851]])


In [76]:
torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)
context_vecs = mha(batch)
print(context_vecs)
print("context_vecs shape:", context_vecs.shape)

keys shape: torch.Size([2, 2, 6, 1])
queries shape: torch.Size([2, 2, 6, 1])
keys transpose shape: torch.Size([2, 2, 1, 6])
tensor([[[0.3190, 0.4858],
         [0.2962, 0.4010],
         [0.2867, 0.3666],
         [0.2702, 0.3928],
         [0.2645, 0.3972],
         [0.2580, 0.4065]],

        [[0.3190, 0.4858],
         [0.2962, 0.4010],
         [0.2867, 0.3666],
         [0.2702, 0.3928],
         [0.2645, 0.3972],
         [0.2580, 0.4065]]], grad_fn=<ViewBackward0>)
context_vecs shape: torch.Size([2, 6, 2])


In [ ]:

gpt2_mha = MultiHeadAttention(d_in = d_in, d_out = 768, context_length = context_length, dropout = 0.0, num_heads = 12)
context_vecs = gpt2_mha(batch)
print(context_vecs)
print("context_vecs shape:", context_vecs.shape)

keys shape: torch.Size([2, 12, 6, 64])
queries shape: torch.Size([2, 12, 6, 64])
keys transpose shape: torch.Size([2, 12, 64, 6])
tensor([[[-2.6138e-03,  7.6756e-02,  8.5986e-02,  ..., -5.1693e-01,
          -1.3521e-01,  1.6488e-01],
         [-1.3610e-01,  3.9011e-02,  8.6464e-02,  ..., -5.7876e-01,
          -2.0540e-01,  4.6149e-02],
         [-1.8881e-01,  2.3534e-02,  8.7422e-02,  ..., -6.0477e-01,
          -2.3375e-01, -4.0457e-04],
         [-1.9569e-01,  1.1563e-02,  8.5370e-02,  ..., -5.4537e-01,
          -2.1686e-01, -1.7504e-02],
         [-1.7868e-01,  3.3283e-02,  5.6790e-02,  ..., -5.0214e-01,
          -2.0144e-01, -3.8340e-02],
         [-1.9644e-01,  9.5950e-03,  7.1341e-02,  ..., -4.9681e-01,
          -2.0249e-01, -3.8925e-02]],

        [[-2.6138e-03,  7.6756e-02,  8.5986e-02,  ..., -5.1693e-01,
          -1.3521e-01,  1.6488e-01],
         [-1.3610e-01,  3.9011e-02,  8.6464e-02,  ..., -5.7876e-01,
          -2.0540e-01,  4.6149e-02],
         [-1.8881e-01,  2.35

In [ ]:
d_in, d_out = 768, 768
context_length = 1024
gpt2_mha = MultiHeadAttention(d_in = d_in, d_out = d_out, context_length = context_length, dropout = 0.0, num_heads = 12)
